In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv002.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/MICCAI-2023


In [2]:
import sys, os, pathlib
import numpy as np
import xarray as xr
import torch
import matplotlib.pyplot as plt
import seaborn as sns

os.environ['DDEBACKEND'] = 'pytorch'
import deepxde

sys.path.append('..')
%aimport mre_pinn

torch.cuda.is_available()

Using backend: pytorch



True

In [ ]:
%autoreload
cohort = mre_pinn.data.ImagingCohort(['0006'])
cohort.load_images()
cohort.preprocess()
dataset = cohort.to_dataset()
dataset.save_xarrays('../data/NAFLD3')

In [35]:
%autoreload
example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/v2', '0006', anat=True)
example.metadata

Loading ../data/NAFLD/v2/0006/wave.nc
Loading ../data/NAFLD/v2/0006/mre.nc
Loading ../data/NAFLD/v2/0006/mre_mask.nc
Loading ../data/NAFLD/v2/0006/anat.nc
Loading ../data/NAFLD/v2/0006/anat_mask.nc


KeyboardInterrupt: 

In [ ]:
example.anat

In [4]:
example.describe()

,,dtype,count,mean,std,min,25%,50%,75%,max
variable,component,,,,,,,,,
wave,scalar,float64,262144.0,0.000139,0.023588,-0.099609,-0.004492,0.00000,0.004736,0.099609
mre,scalar,int16,262144.0,1505.871937,1335.735833,0.000000,0.000000,1340.00000,2212.000000,14078.000000
mre_mask,scalar,float32,262144.0,0.171467,0.376916,0.000000,0.000000,0.00000,0.000000,1.000000
anat,scalar,float32,5242880.0,153.163483,273.202576,-30.489124,0.015030,31.54632,206.595573,4559.377930


In [5]:
mre_pinn.baseline.eval_direct_baseline(example, frequency=40)
mre_pinn.baseline.eval_fem_baseline(example, frequency=40, hetero=True)

Evaluating direct baseline
Evaluating FEM baseline
100%|██████████| 4/4 [22:10<00:00, 332.62s/it]


In [26]:
%autoreload
pde = mre_pinn.pde.WaveEquation.from_name('hetero', omega=40, detach=True)

In [27]:
%autoreload
pinn = mre_pinn.model.MREPINN(
    example,
    omega=60,
    n_layers=5,
    n_hidden=128,
    polar_input=True
)
pinn

MREPINN(
  (u_pinn): PINN(
    (hidden0): Linear(in_features=6, out_features=128, bias=True)
    (hidden1): Linear(in_features=134, out_features=128, bias=True)
    (hidden2): Linear(in_features=262, out_features=128, bias=True)
    (hidden3): Linear(in_features=390, out_features=128, bias=True)
    (output): Linear(in_features=518, out_features=1, bias=True)
  )
  (mu_pinn): PINN(
    (hidden0): Linear(in_features=6, out_features=128, bias=True)
    (hidden1): Linear(in_features=134, out_features=128, bias=True)
    (hidden2): Linear(in_features=262, out_features=128, bias=True)
    (hidden3): Linear(in_features=390, out_features=128, bias=True)
    (output): Linear(in_features=518, out_features=6, bias=True)
  )
)

In [28]:
%autoreload
model = mre_pinn.training.MREPINNModel(
    example, pinn, pde,
    loss_weights=[1, 0, 1, 1e-16],
    pde_warmup_iters=5000,
    pde_step_iters=5000,
    pde_init_weight=1e-18,
    n_points=1024
)
model.compile(optimizer='adam', lr=1e-4, loss=mre_pinn.training.losses.msae_loss)

Compiling model...
'compile' took 0.000218 s



In [32]:
example.anat

<xarray.DataArray 't1_pre_in' (component: 5, x: 256, y: 256, z: 16)>
[5242880 values with dtype=float32]
Coordinates:
  * x          (x) float64 -0.4138 -0.411 -0.4082 ... 0.2978 0.3006 0.3034
  * y          (y) float64 -0.3917 -0.3889 -0.3861 ... 0.3198 0.3227 0.3255
  * z          (z) float64 -0.1589 -0.1336 -0.1083 ... 0.17 0.1953 0.2206
  * component  (component) object 't1_pre_in' 't1_pre_out' ... 't1_pre_fat' 't2'
    region     (x, y, z) float32 nan nan nan nan nan nan ... nan nan nan nan nan

In [10]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
model.benchmark(100)

# iterations: 100


IndexError: The shape of the mask [262144] at index 0 does not match the shape of the indexed tensor [327680, 16] at index 0

In [30]:
for t in model.data.get_raw_tensors(device='cpu', patch_size=1):
    print(t.dtype, t.shape)

torch.float32 torch.Size([262144, 3])
torch.float64 torch.Size([262144, 1])
torch.int16 torch.Size([262144, 1])
torch.bool torch.Size([262144])
torch.float32 torch.Size([1048576, 5])


In [ ]:
test_eval = mre_pinn.testing.TestEvaluator(test_every=100, interact=True)
test_eval.model = model
test_eval.test()

In [ ]:
model.train(100000, callbacks=[test_eval])